# Robustness of Crayfish Nervous System to Environmental pH
## Step 1: Format Data

In [1]:
import neo, os
import numpy as np
from mne import create_info
from mne.io import RawArray

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
stop_list = ['171208_pH74_1_1.abf', '171208_pH72_1_1.abf']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main body.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Locate files.
files = [f for f in os.listdir('raw') if f.endswith('abf')]
files = [f for f in files if f not in stop_list]

for f in files:

    ## Load recordings.
    recordings, = neo.AxonIO(filename='raw/%s' %f).read_block().segments
    signal, = recordings.analogsignals

    ## Concatenate raw recordings.
    data = np.hstack([np.asarray(signal, dtype=np.float64)  * 1e-6]).T
    if f.startswith('171212'): data *= -1 # Fix recording issue on Day 2.
    
    ## Create info object.
    sfreq = float(signal.sampling_rate)
    ch_names = ['nerve']
    ch_types = 'bio'

    info = create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

    ## Create Raw object.
    raw = RawArray(data, info, verbose=False)
    
    ## Save Raw object.  
    raw.save('raw/%s' %f.replace('.abf','_raw.fif'), overwrite=True, verbose=False)
    
print('Done.')

Done.


In [2]:
f = '171208_pH74_1_1.abf'

## Load recordings.
recordings, = neo.AxonIO(filename='raw/%s' %f).read_block().segments
signal, = recordings.analogsignals

## Concatenate raw recordings.
data = np.hstack([np.asarray(signal, dtype=np.float64)  * 1e-6]).T
if f.startswith('171212'): data *= -1 # Fix recording issue on Day 2.

## Create info object.
sfreq = float(signal.sampling_rate)
ch_names = ['nerve']
ch_types = 'bio'

info = create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

## Create Raw object.
raw = RawArray(data, info, verbose=False)

## Step 2: Preprocess Data

In [3]:
import os
import numpy as np
from mne import Epochs, make_fixed_length_events
from mne.io import Raw
from pandas import DataFrame, concat
from spike_sorting import find_threshold, peak_finder

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sessions = ['171208', '171212']

## Filter parameters.
l_freq = 300
h_freq = 3000

## Epoching.
duration = 9.99 # seconds

## Spike detection parameters.
k = 5
reject = 200 # uV

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

spikes = []
for session in sessions:
    
    ## Locate files.
    files = sorted([f for f in os.listdir('raw') if f.startswith(session) and f.endswith('fif')])
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Assemble recordings.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for f in files:
    
        ## Extract metadata.
        session, pH, _, recording, _ = f.split('_')
        pH = float(pH[-2:]) / 10
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Load and prepare data.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Load raw.
        raw = Raw('raw/%s' %f, preload=True, verbose=False)
        
        ## Filter data.
        raw = raw.filter(l_freq, h_freq, picks=[0], method='fir', phase='zero', 
                         fir_design='firwin', verbose=False)
        
        ## Crop raw (remove filter artifact).
        raw = raw.crop(0.05)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Epoching.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Make events of equal length.
        events = make_fixed_length_events(raw, 1, duration=duration)
        
        ## Make epochs.
        epochs = Epochs(raw, events, tmin=0, tmax=duration, baseline=None,
                        picks=[0], preload=True, verbose=False)
        
        ## Extract data.
        times = epochs.times
        epochs = epochs.get_data().squeeze()
        epochs *= 1e6 # Convert to uV.

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Spike detection.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Set threshold.
        threshold = find_threshold(epochs, k)
        
        ## Iterate over epochs.
        for i, epoch in enumerate(epochs):
            
            ## Detect spikes.
            peak_loc, peak_mag = peak_finder(epoch, threshold)
            if np.any(peak_loc): peak_loc = times[peak_loc] + i * times.max()
    
            ## Store as DataFrame. Append.
            df = DataFrame( np.vstack([peak_loc, peak_mag]).T, columns=('Time','Amplitude') )
            for column, value in zip(['Epoch','Recording','pH','Session'], [i+1, recording, pH, session]):
                df.insert(0, column, value)
            spikes.append(df)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Assemble Spikes DataFrame.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Concatenate spike DataFrames.
spikes = concat(spikes)

## Amplitude rejection.
spikes = spikes[spikes.Amplitude < reject]
    
## Save.
spikes.to_csv('spikes.csv', index=False)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Assemble Counts DataFrame.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Setup count GroupBy object.
columns = ['Session','pH','Recording','Epoch']
gb = spikes.groupby(columns)

## Compute counts.
counts = gb.Amplitude.count().reset_index()
counts.columns = columns + ['Count']

## Convert to Frequency.
counts['Frequency'] = counts['Count'] / duration

## Compute average spike amplitude.
amplitude = gb.Amplitude.mean().reset_index()

## Merge. Save.
counts = counts.merge(amplitude, on=columns)

## Save DataFrames.
counts.to_csv('counts.csv', index=False)

print('Done.')

Done.


### Sanity Checks

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mne.io import Raw
from spike_sorting import find_threshold, peak_finder
sns.set_style('white')
sns.set_context('notebook', font_scale=2)
%matplotlib inline

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
files = ['171212_ph%s_1_%s_raw.fif' %(pH,recording) for pH in [55,65,75] 
         for recording in np.arange(5)+1]

## Filter parameters.
l_freq = 300
h_freq = 3000

## Spike detection parameters.
k = 5

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for f in files:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load raw.
    raw = Raw('raw/%s' %f, preload=True, verbose=False)

    ## Filter data.
    raw = raw.filter(l_freq, h_freq, picks=[0], method='fir', phase='zero', 
                     fir_design='firwin', verbose=False)

    ## Crop raw (remove filter artifact).
    raw = raw.crop(0.05)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Initialize canvas.
    fig, axes = plt.subplots(3,1,figsize=(12,12))
    
    ## Define times for plotting.
    tmax = raw.times.max()
    time_limits = [(0,1), (tmax/2 - 0.5, tmax/2 + 0.5), (tmax-1,tmax)]
    
    ## Define threshold.
    threshold = find_threshold(raw.get_data() * 1e6, k)
    
    for ax, tlim in zip(axes, time_limits):
    
        ## Extract data.
        data, times = raw.get_data(start=raw.time_as_index(tlim[0])[0], 
                                   stop=raw.time_as_index(tlim[1])[0],
                                   return_times=True)
        data = data.squeeze() * 1e6
        
        ## Find spikes.
        peak_loc, _ = peak_finder(data, threshold)
        
        ## Plot.
        ax.plot(times, data, lw=2)
        ax.hlines(threshold, *tlim)
        ax.set(xlim=tlim, xlabel='Time (s)', ylim=(-100,100), ylabel=r'Amplitude ($\mu V$)')
        ax.set_title('%s spikes' %peak_loc.size, fontsize=24)
        
    ## Save.
    sns.despine()
    plt.tight_layout()
    plt.savefig('qc/%s.png' %f.replace('_raw.fif', '.png'), dpi=180)
    plt.close('all')
    
print('Done.')

Done.


## Step 3: Statistics and Visualizations

### Spiking Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv
from statsmodels.api import OLS
sns.set_style('white')
sns.set_context('notebook', font_scale=2)
%matplotlib inline

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

counts = read_csv('counts.csv')
counts['log_freq'] = np.log10(counts.Frequency)
s1, s2 = counts[counts.Session==171208], counts[counts.Session==171212]

In [ ]:

result = OLS.from_formula('log_freq ~ pH', data=s1).fit()
print(result.summary2())

x1, x2 = s1.pH.min(), s1.pH.max()
x = np.linspace(x1,x2,100)
a, b = result.params
A = np.power(10, a)
y = A * np.power(10, b * x)
plt.plot(x,y)
plt.scatter(s1.pH, s1.Frequency, s=25, color='none', edgecolor='k')

In [ ]:

result = OLS.from_formula('log_freq ~ pH', data=s2).fit()
print(result.summary2())

x = np.linspace(2.5,7.5,100)
a, b = result.params
A = np.power(10, a)
y = A * np.power(10, b * x)
plt.plot(x,y)
plt.scatter(s2.pH, s2.Frequency, s=25, color='none', edgecolor='k')

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(12,8))

## Plot Session 1.
for ax, session, palette in zip(axes, counts.Session.unique(), ['Blues','Greens']):
    sns.boxplot('pH', 'Frequency', data=counts[counts.Session==session], 
                palette=sns.color_palette(palette), ax=ax)
    sns.swarmplot('pH', 'Frequency', data=counts[counts.Session==session], 
                  size=3, color='0.3', ax=ax)

sns.despine()
plt.tight_layout()

In [ ]:
spikes = read_csv('spikes.csv')
fig, axes = plt.subplots(2,6,figsize=(24,6), sharex=True, sharey=True)

for i, session in enumerate(spikes.Session.unique()):
    
    for j, pH in enumerate(spikes.loc[spikes.Session==session,'pH'].unique()):
        
        ix = np.logical_and(spikes.Session==session, spikes.pH==pH)
        sns.distplot(spikes.loc[ix, 'Amplitude'], kde=False, ax=axes[i,j])
        
axes[0,0].set(xlim=(18,100),xlabel='')

sns.despine()
plt.tight_layout()